In [1]:
from utility_funcs import get_train_labels_test

import numpy as np
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm


In [2]:
train, labels, test = get_train_labels_test()

In [3]:
features = [f for f in train.columns if 'sensor' in f]

In [4]:
groups = train["sequence"]
train = train.drop(["subject", "step",'sequence'], axis=1)
test = test.drop([ "subject", "step",'sequence'], axis=1)

In [6]:
length = len(train)
train_size = int(length * 0.85) - int((length * 0.85) % 60)
test_size = length - train_size
length_y = len(labels)
train_size_y = int(length_y * 0.85)
test_size_y = length_y - train_size_y
X_train, X_val = train[0:train_size], train[train_size:length]
y_train, y_val = labels['state'][0:train_size_y], labels['state'][train_size_y:length_y]
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(1324320, 13) (22072,)
(233760, 13) (3896,)


In [7]:
y_train = np.array(y_train)
y_valid = np.array(y_val)

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train = scaler.fit_transform(X_train)
valid = scaler.transform(X_val)
test = scaler.transform(test)


In [9]:
class CustomDataset(Dataset):
    def __init__(self,X, sequence_len, y = None, mode = 'train'):
        self.data = X
        self.target = y
        self.sequence_len = sequence_len
        self.mode = mode
    
    def __len__(self):
        return (self.data.shape[0]//60) # each sequence is of length 60 
    
    def __getitem__(self, idx):
        out_x = self.data[idx]
        if self.mode == 'train':
            out_y =  self.target[idx[0]//self.sequence_len]
            return out_x, out_y
        else: 
            return out_x


def CustomDataloader(dataset,  dataset_num, sequence_len,  input_size, batch_size):
    sampler = np.array([list(range(i*sequence_len, (i+1)*sequence_len)) for i in range(dataset_num//sequence_len)])
    dataloader = DataLoader(dataset, batch_size, sampler=sampler)
    return dataloader

In [10]:
seq_length = 60
train_dataset = CustomDataset(train, sequence_len=seq_length, y=y_train)
valid_dataset = CustomDataset(valid, sequence_len=seq_length, y=y_val)
test_dataset = CustomDataset(test, sequence_len=seq_length, mode="test")

train_dataloader = CustomDataloader(
    train_dataset, train.shape[0], seq_length, train.shape[1], 512
)
valid_dataloader = CustomDataloader(
    valid_dataset, valid.shape[0], seq_length, valid.shape[1], 512
)
test_dataloader = CustomDataloader(
    test_dataset, test.shape[0], seq_length, test.shape[1], 512
)


In [11]:
valid.shape[0],valid.shape[1]

(233760, 13)

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [13]:
class LSTM(nn.Module):
    def __init__(self,input_size,num_classes,hidden_size,num_layers):
        super().__init__()
        
        self.lstm1 = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True,dropout=0,bidirectional=True)
        
        self.lstm2 = nn.LSTM(hidden_size*2,hidden_size*2,num_layers,batch_first=True,dropout=0,bidirectional=True)
        
        self.lstm3= nn.LSTM(hidden_size*4,hidden_size*4,num_layers,batch_first=True,dropout=0,bidirectional=False)

        self.final = nn.Sequential(
            nn.ReLU(),
            nn.Linear(hidden_size*60*4, num_classes),
        )
            
    def forward(self,x):
        
        out1, _ = self.lstm1(x)
        
        out2, _ = self.lstm2(out1)
        
        out3, _ = self.lstm3(out2)
        
        out3 = out3.reshape(out3.shape[0],-1)
        
        out4 = self.final(out3)
        return out4

In [14]:
input_size = len(features)
hidden_size = 64
num_layers = 2
num_classes = 1
learning_rate = 1e-3
num_epochs = 500

model = LSTM(input_size,num_classes,hidden_size,num_layers)
model.to(device)

LSTM(
  (lstm1): LSTM(13, 64, num_layers=2, batch_first=True, bidirectional=True)
  (lstm2): LSTM(128, 128, num_layers=2, batch_first=True, bidirectional=True)
  (lstm3): LSTM(256, 256, num_layers=2, batch_first=True)
  (final): Sequential(
    (0): ReLU()
    (1): Linear(in_features=15360, out_features=1, bias=True)
  )
)

In [15]:

criterion = nn.BCEWithLogitsLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
num_warmup_steps = int(0.1 * num_epochs * len(train_dataloader))
num_training_steps = int(num_epochs * len(train_dataloader))
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps, num_training_steps
)

In [16]:
torch.cuda.empty_cache()
for epoch in tqdm(range(num_epochs-1)): 
    model.train()
    for trainX, train_y in train_dataloader:
        outputs = model(trainX.to(device,dtype=torch.float32)).squeeze(-1)
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        loss = criterion(outputs, train_y.to(device,dtype=torch.float32))
        loss.backward()

        optimizer.step()
        scheduler.step()

    model.eval()
    for validX, valid_y in valid_dataloader:
        with torch.no_grad():
            val_out = model(validX.to(device,dtype=torch.float32)).squeeze(-1)
            vall_loss = criterion(val_out,valid_y.to(device,dtype=torch.float32))

    if epoch % 10 == 0:
        
          print("Epoch: %d, loss: %1.5f valid loss:  %1.5f " %(epoch, loss.cpu().item(),vall_loss.cpu().item()))

  0%|          | 0/499 [00:08<?, ?it/s]


KeyError: 0